In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titaniccap8/train.csv
/kaggle/input/titaniccap8/test.csv
/kaggle/input/titaniccap8/gender_submission.csv


**Bibliotecas**

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
np.random.seed(0)

**Reutilizando o notebook do capítulo 7, somente os trechos iniciais:**

In [23]:
train_data = pd.read_csv("/kaggle/input/titaniccap8/train.csv")
test_data = pd.read_csv("/kaggle/input/titaniccap8/test.csv")


**1. Será necessário realizar o encoding das variáveis categóricas, no momento, 3
 estratégias que serão utilizadas são o one-hot, label encoding e ordinal encoding.
 Para fazer isso será necessário relembrar quais são as features categóricas, definir a
 estratégia que será utilizada, criar as features codificadas e retirar as categóricas:**

**a. Utilizar o método “.info( )” para relembrar quais são as features categóricas;**

In [24]:
train_data.info( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [25]:
test_data.info( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


**b. Utilizar o método “.select_dtypes” para identificar o nome das colunas
 numéricas e categóricas, para numéricas identifique formatos “int64” e
 “float64”, para categóricas identifique “Object”;**

**Train_data:**

In [26]:
# Colunas numéricas
num_cols = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Colunas Numéricas:", num_cols)

# Colunas categóricas
cat_cols = train_data.select_dtypes(include=['object']).columns.tolist()
print("Colunas Categóricas:", cat_cols)

Colunas Numéricas: ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Colunas Categóricas: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


**Test_data:**

In [27]:
# Colunas numéricas
num_cols = test_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Colunas Numéricas:", num_cols)

# Colunas categóricas
cat_cols = test_data.select_dtypes(include=['object']).columns.tolist()
print("Colunas Categóricas:", cat_cols)

Colunas Numéricas: ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Colunas Categóricas: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


**c. Temos algumas features categóricas que contém uma ordem clara, ou seja,
 existe o primeiro valor, segundo valor, terceiro valor, assim por diante, já em
 outras, isso não acontece.**
 
 **i.  Para as features que possuem ordem, o label encoder é mais indicado
 como primeira abordagem;**
 
 **ii.Para features sem ordem definida, one-hot encoding pode ser a melhor
 opção;**

**Letra i:**

**tain_data:**

In [28]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data['Pclass_encoded'] = le.fit_transform(train_data['Pclass'])
train_data[['Pclass', 'Pclass_encoded']].head(10)

,Pclass,Pclass_encoded
0,3,2
1,1,0
2,3,2
3,1,0
4,3,2
5,3,2
6,1,0
7,3,2
8,3,2
9,2,1


**test_data:**

In [29]:
test_data['Pclass_encoded'] = le.fit_transform(test_data['Pclass'])
test_data[['Pclass', 'Pclass_encoded']].head(10)

,Pclass,Pclass_encoded
0,3,2
1,3,2
2,2,1
3,3,2
4,3,2
5,3,2
6,3,2
7,2,1
8,3,2
9,3,2


**letra ii:**

**train_data:**

In [30]:
# 1) Deck antes do drop
train_data['Deck'] = train_data['Cabin'].str[0].fillna('U')
train_data.loc[train_data['Deck'] == 'T', 'Deck'] = 'A'
deck_order = {'U': 0, 'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A': 7}
train_data['Deck_encoded'] = train_data['Deck'].map(deck_order).fillna(0).astype(int)

# Agora pode remover 'Cabin' e 'Deck'
train_data.drop(columns=['Cabin', 'Deck'], inplace=True)

# One-hot apenas nas colunas que ainda existem
# Sex
encoder_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_sex = encoder_sex.fit_transform(train_data[['Sex']])
sex_cols = encoder_sex.get_feature_names_out(['Sex'])
train_data = pd.concat(
    [train_data.drop(columns=['Sex']),
     pd.DataFrame(encoded_sex, columns=sex_cols, index=train_data.index)],
    axis=1
)

# Embarked
encoder_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_embarked = encoder_embarked.fit_transform(train_data[['Embarked']])
embarked_cols = encoder_embarked.get_feature_names_out(['Embarked'])
train_data = pd.concat(
    [train_data.drop(columns=['Embarked']),
     pd.DataFrame(encoded_embarked, columns=embarked_cols, index=train_data.index)],
    axis=1
)

NameError: name 'OneHotEncoder' is not defined

**test_data:**

In [ ]:
# Deck no test antes do drop
test_data['Deck'] = test_data['Cabin'].str[0].fillna('U')
test_data.loc[test_data['Deck'] == 'T', 'Deck'] = 'A'
test_data['Deck_encoded'] = test_data['Deck'].map(deck_order).fillna(0).astype(int)
test_data.drop(columns=['Cabin','Deck'], inplace=True)

# Reutilize encoders treinados no train
encoded_sex_test = encoder_sex.transform(test_data[['Sex']])
test_data = pd.concat(
    [test_data.drop(columns=['Sex']),
     pd.DataFrame(encoded_sex_test, columns=sex_cols, index=test_data.index)],
    axis=1
)

encoded_embarked_test = encoder_embarked.transform(test_data[['Embarked']])
test_data = pd.concat(
    [test_data.drop(columns=['Embarked']),
     pd.DataFrame(encoded_embarked_test, columns=embarked_cols, index=test_data.index)],
    axis=1
)

In [ ]:
# Garante que test_data tenha as mesmas colunas do train_data
missing_cols = set(train_data.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0

# Reordena colunas na mesma ordem do train_data
test_data = test_data[train_data.columns]

**d. Criar uma função que realize o one-hot encode e, como saída, retorna um
 novo dataframe com as colunas que resultam da codificação, devidamente
 nomeadas, ao invés das features categóricas:**

  **i.Os argumentos deverão ser: o dataframe que será manipulado e as
 colunas que serão codificadas;**

 **ii.Realizar uma cópia do banco de dados que foi dado como argumento;**

 **iii.Criar um loop que realize o encoding das colunas presentes no
 argumento da função, as features resultantes devem conter o nome do
 valor que representam (OBS: utilizar o método “.get_feature_names”);**

 **iv.Ainda dentro do loop, retirar do dataframe copiado as features
 categóricas e mesclar as features criadas pelo one-hot encoder;** 

**Para o train_data:**

In [ ]:
from typing import List

from sklearn.preprocessing import OneHotEncoder

def one_hot_encode_df(_input_df: pd.DataFrame, cols_to_encode: List[str]) -> pd.DataFrame:
    """
    Codifica colunas categóricas usando OneHotEncoder e retorna o DataFrame com as colunas codificadas substituindo as originais.
    """

    # ii. Cópia do DataFrame
    train_data = _input_df.copy()

    # Verificação simples para evitar erros
    missing = [col for col in cols_to_encode if col not in train_data.columns]
    if missing:
        raise ValueError(f"Colunas não encontradas no DataFrame: {missing}")

    # iii. Loop para codificar colunas
    for col in cols_to_encode:
        ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        encoded = ohe.fit_transform(train_data[[col]])

        # Obtem nomes das novas features codificadas
        try:
            new_features = ohe.get_feature_names([col])
        except AttributeError:
            new_features = ohe.get_feature_names_out([col])

        # Cria DataFrame das colunas codificadas
        encoded_df = pd.DataFrame(encoded, columns=new_features, index=train_data.index)
        encoded_df = encoded_df.astype(int)

        # iv. Remove a coluna original e insere as codificadas
        train_data.drop(columns=[col], inplace=True)
        train_data = pd.concat([train_data, encoded_df], axis=1)

    return train_data

**Para o test_data:**

In [ ]:
def one_hot_encode_df(_input_df: pd.DataFrame, cols_to_encode: List[str]) -> pd.DataFrame:
    """
    Codifica colunas categóricas usando OneHotEncoder e retorna o DataFrame com as colunas codificadas substituindo as originais.
    """

    # ii. Cópia do DataFrame
    test_data = _input_df.copy()

    # Verificação simples para evitar erros
    missing = [col for col in cols_to_encode if col not in test_data.columns]
    if missing:
        raise ValueError(f"Colunas não encontradas no DataFrame: {missing}")

    # iii. Loop para codificar colunas
    for col in cols_to_encode:
        ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        encoded = ohe.fit_transform(test_data[[col]])

        # Obtem nomes das novas features codificadas
        try:
            new_features = ohe.get_feature_names([col])
        except AttributeError:
            new_features = ohe.get_feature_names_out([col])

        # Cria DataFrame das colunas codificadas
        encoded_df = pd.DataFrame(encoded, columns=new_features, index=test_data.index)
        encoded_df = encoded_df.astype(int)

        # iv. Remove a coluna original e insere as codificadas
        test_data.drop(columns=[col], inplace=True)
        test_data = pd.concat([test_data, encoded_df], axis=1)

    return test_data

**Exemplo de uso do código:**

**Escolha as colunas categóricas que quer utilizar:**

cols_para_codificar = ['Sex', 'Embarked']

**Essas colunas serão transformadas em várias colunas binárias (0 ou 1) representando cada categoria**

**Chame a função que você definiu**

train_data_encoded = one_hot_encode_df(train_data, cols_para_codificar)

**Visualize o resultado:**

print(train_data_encoded.head())


**e. Criar novas features para realizar o label encoding das features que contém
 um ordenamento claro, e depois retirar as features categóricas que foram
 codificadas.**

**Para o train_data:**

In [ ]:
# 🔹 1. Extraia o Title a partir de Name
train_data['Title'] = (
    train_data['Name']
    .str.extract(r",\s*([^\.]+)\.", expand=False)
    .str.strip()
)

# 🔹 2. Agrupe títulos raros
title_map = {
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
    'Lady': 'Nobility', 'Countess': 'Nobility', 'Dona': 'Nobility',
    'Sir': 'Nobility', 'Jonkheer': 'Nobility', 'Don': 'Nobility',
    'Rev': 'Officer', 'Col': 'Officer', 'Major': 'Officer',
    'Dr': 'Officer', 'Capt': 'Officer'
}
train_data['Title'] = train_data['Title'].replace(title_map)

# 🔹 3. Codificação ordinal do Title
title_order = {
    'Master': 0, 'Miss': 1, 'Mrs': 2,
    'Mr': 3, 'Officer': 4, 'Nobility': 5
}
train_data['Title_encoded'] = train_data['Title'].map(title_order).fillna(-1).astype(int)

# 🔹 4. Extraia o Deck a partir de Cabin — se existir!
if 'Cabin' in train_data.columns:
    # Evite astype(str) para não transformar NaN em 'nan'
    train_data['Deck'] = train_data['Cabin'].str[0].fillna('U')
    train_data.loc[train_data['Deck'] == 'T', 'Deck'] = 'A'

    deck_order = {'U': 0, 'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A': 7}
    train_data['Deck_encoded'] = train_data['Deck'].map(deck_order).fillna(0).astype(int)
else:
    # Se não houver Cabin, ainda garanta a coluna codificada
    train_data['Deck_encoded'] = 0  # tudo desconhecido

# 🔹 5. Remova colunas temporárias (sem quebrar)
train_data.drop(columns=['Title', 'Deck'], errors='ignore', inplace=True)

In [ ]:
# 4) Checagem rápida
train_data[['PassengerId', 'Title_encoded', 'Deck_encoded']].head(10)

**Para o test_data:**

In [ ]:
# 🔹 1. Extraia o Title a partir de Name
test_data['Title'] = (
    test_data['Name']
    .str.extract(r",\s*([^\.]+)\.", expand=False)
    .str.strip()
)

# 🔹 2. Agrupe títulos raros
title_map = {
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
    'Lady': 'Nobility', 'Countess': 'Nobility', 'Dona': 'Nobility',
    'Sir': 'Nobility', 'Jonkheer': 'Nobility', 'Don': 'Nobility',
    'Rev': 'Officer', 'Col': 'Officer', 'Major': 'Officer',
    'Dr': 'Officer', 'Capt': 'Officer'
}
test_data['Title'] = test_data['Title'].replace(title_map)

# 🔹 3. Codificação ordinal do Title
title_order = {
    'Master': 0, 'Miss': 1, 'Mrs': 2,
    'Mr': 3, 'Officer': 4, 'Nobility': 5
}
test_data['Title_encoded'] = test_data['Title'].map(title_order).fillna(-1).astype(int)

# 🔹 4. Extraia o Deck a partir de Cabin — se existir!
if 'Cabin' in test_data.columns:
    test_data['Deck'] = test_data['Cabin'].str[0].fillna('U')
    test_data.loc[test_data['Deck'] == 'T', 'Deck'] = 'A'

    deck_order = {'U': 0, 'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A': 7}
    test_data['Deck_encoded'] = test_data['Deck'].map(deck_order).fillna(0).astype(int)
else:
    test_data['Deck_encoded'] = 0  # tudo desconhecido

# 🔹 5. Remova colunas temporárias (sem quebrar)
test_data.drop(columns=['Title', 'Deck'], errors='ignore', inplace=True)

# 🔹 Checagem rápida
test_data[['PassengerId', 'Title_encoded', 'Deck_encoded']].head(10)

**2. Agora será realizado um novo treinamento e avaliação de modelo, os passos serão os
 mesmo realizados no item 6.1 g), mas agora com o banco de dados que é resultado
 do encoding das variáveis categóricas. Após essa avaliação, será possível notar a
 evolução da precisão do modelo produzido utilizando essas novas técnicas.**

**Train_data:**

In [ ]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import pandas as pd

# 1) Separe X e y
drop_cols = ['Survived', 'Name', 'Ticket']  # Name/Ticket são texto livre
X = train_data.drop(columns=drop_cols, errors='ignore')
y = train_data['Survived']

# 2) Garanta que X não tenha colunas de texto restantes
obj_cols = X.select_dtypes(include=['object']).columns.tolist()
if obj_cols:
    print(f"Removendo colunas categóricas remanescentes de X (não numéricas): {obj_cols}")
    X = X.drop(columns=obj_cols)

# 3) Imputação de valores ausentes (mediana para numéricos)
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)

# (opcional) Checagem rápida de NaNs
assert not X_imputed.isna().any().any(), "Ainda há NaN após a imputação!"

# 4) Split
X_train, X_valid, y_train, y_valid = train_test_split(
    X_imputed, y, test_size=0.25, random_state=0, stratify=y
)

# 5) Treinar modelo
model = RandomForestClassifier(
    n_estimators=300,
    random_state=0,
    n_jobs=-1
)
model.fit(X_train, y_train)

# 6) Avaliar
y_pred = model.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
print("✅ Acurácia com imputação e apenas numéricos:", round(acc, 4))

**test_data:**

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# 1) Preparar X (numérico) e y a partir do train_data
drop_cols = ['Survived', 'Name', 'Ticket']
X_train_full = train_data.drop(columns=drop_cols, errors='ignore').copy()
y = train_data['Survived'].copy()

# Remover quaisquer colunas de texto remanescentes (garantir somente numéricos)
obj_cols_train = X_train_full.select_dtypes(include=['object']).columns.tolist()
if obj_cols_train:
    print(f"Removendo colunas categóricas remanescentes do train: {obj_cols_train}")
    X_train_full = X_train_full.drop(columns=obj_cols_train)

# Guardar ordem de colunas do treino para alinhar o test depois
train_cols = X_train_full.columns.tolist()

# 2) Imputação (ajusta no train e reusa no test)
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train_full),
    columns=train_cols,
    index=X_train_full.index
)
assert not X_train_imputed.isna().any().any(), "Ainda há NaN após a imputação no treino!"

# 3) Treinar modelo (novo objeto, para não depender de 'model' já existente)
rf_model = RandomForestClassifier(
    n_estimators=300,
    random_state=0,
    n_jobs=-1
)
rf_model.fit(X_train_imputed, y)

# 4) Preparar o test_data de forma consistente
X_test = test_data.drop(columns=['Name', 'Ticket'], errors='ignore').copy()

# Remover quaisquer colunas de texto remanescentes
obj_cols_test = X_test.select_dtypes(include=['object']).columns.tolist()
if obj_cols_test:
    print(f"Removendo colunas categóricas remanescentes do test: {obj_cols_test}")
    X_test = X_test.drop(columns=obj_cols_test)

# 5) Alinhar colunas do test ao conjunto de colunas do train
#    - adiciona colunas que faltam com 0
#    - remove colunas extras
X_test_aligned = X_test.reindex(columns=train_cols, fill_value=0)

# 6) Imputar usando o MESMO imputer treinado no treino (transform, não fit_transform)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test_aligned),
    columns=train_cols,
    index=X_test_aligned.index
)
assert not X_test_imputed.isna().any().any(), "Ainda há NaN após a imputação no test!"

# 7) Prever no test_data
y_test_pred = rf_model.predict(X_test_imputed)

# 8) Se existir rótulo em test_data, calcular acurácia; caso contrário, mostrar amostra e/ou salvar CSV
if 'Survived' in test_data.columns:
    from sklearn.metrics import accuracy_score
    test_acc = accuracy_score(test_data['Survived'], y_test_pred)
    print("✅ Acurácia no test_data:", round(test_acc, 4))
else:
    print("🔮 Previsões geradas para test_data (primeiros 10):", y_test_pred[:10])
    # Exemplo de arquivo de submissão (Kaggle):
    # submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_test_pred})
    # submission.to_csv('submission.csv', index=False)
    # print("💾 Arquivo 'submission.csv' salvo.")


**8.2- Pipelines**

 **1. Deverá ser criada uma função que produz e avalia pipelines que utilizam diversas
 estratégias de imputing de valores nulos e encoding de variáveis categóricas.**
 
 **a. Colocar de volta os valores nulos das colunas “Age” e “Embarked” do
 dataframe de treino, utilizando as colunas do banco de dados original.
 Lembrando que, ao iniciar este notebook, foi feita uma cópia deste banco de
 dados.**

In [ ]:
def restore_columns_from_original(
    df_work: pd.DataFrame,
    df_original_copy: pd.DataFrame,
    cols_to_restore=("Age", "Embarked"),
    key_col="PassengerId",
) -> pd.DataFrame:
    """
    Substitui em df_work as colunas indicadas pelos valores da cópia original,
    alinhando pela chave key_col. Mantém o restante do df_work inalterado.
    """
    # Checagens básicas
    if key_col not in df_work.columns or key_col not in df_original_copy.columns:
        raise ValueError(f"Chave '{key_col}' não encontrada em ambos os dataframes.")
    for c in cols_to_restore:
        if c not in df_original_copy.columns:
            raise ValueError(f"Coluna '{c}' ausente no df_original_copy.")

    # Alinha por chave e substitui as colunas
    restored = df_work.merge(
        df_original_copy[[key_col, *cols_to_restore]],
        on=key_col,
        how="left",
        suffixes=("", "__orig")
    )
    # Copia de volta os valores originais para as colunas-alvo
    for c in cols_to_restore:
        restored[c] = restored[f"{c}__orig"]
        restored.drop(columns=[f"{c}__orig"], inplace=True)

    return restored

# Exemplo de uso (ajuste nomes conforme seu notebook):
# df_train = restore_columns_from_original(df_train, df_train_original, cols_to_restore=("Age", "Embarked"), key_col="PassengerId")

# Verificação rápida:
# df_train[["Age", "Embarked"]].isna().sum()

**b. Criar uma função que produza e avalie pipelines que utilizam estratégias
 indicadas nos argumentos:**
 
 **i.Argumentos:**
 
 **● data: banco de dados a partir do qual será produzido o modelo;**
 
 **● encoder: a estratégia de encoding de variáveis categóricas,
 como one-hot ou label encoder;**

 **● model: o algoritmo que irá produzir o modelo, inicialmente será
 o random forest classifier;**
 
 **● numerical_imputer: a estratégia usada para substituir valores
 nulos nas features numéricas, como "mean" ou "median";**
 
 **● categorical_imputer: estratégia para preencher valores nulos
 nas features categóricas, aqui será usada apenas "mostfrequent", que é substituir pelo valor que mais aparece no bancode dados;**
 
 **ii. Definir as features e o target que serão usados para treinar o modelo e
 separar o banco de dados utilizando o train-test split.**
 
 **iii. Criar um preprocessor que: para as features numéricas, utilize o
 imputer indicado por “numerical_imputer”; para as categóricas, utilize o
 imputer fornecido em “categorical_imputer” e como encoder utilize o
 argumento “encoder”.**
 
 **iv. Produzir pipeline utilizando o preprocessor previamente construído e o
 modelo do argumento “model”.**
 
 **v. Realizar o treinamento da pipeline, gerar predições e avaliar utilizando
 accuracy score.**
 
 **vi. A função deve retornar o resultado da avaliação em porcentagem.**

In [ ]:
from typing import Union, Iterable, Optional

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

def _make_ohe():
    # Compatível com versões diferentes do scikit-learn
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

def train_eval_pipeline(
    data: pd.DataFrame,
    encoder: str = "onehot",  # "onehot" ou "label"
    model=None,               # ex.: RandomForestClassifier(...)
    numerical_imputer: str = "median",      # "mean" | "median" | "most_frequent" | "constant"
    categorical_imputer: str = "most_frequent",  # para este exercício, manter "most_frequent"
    target_col: str = "Survived",
    id_cols: Optional[Iterable[str]] = ("PassengerId",),
    test_size: float = 0.2,
    random_state: int = 42,
) -> float:
    """
    Produz e avalia uma pipeline conforme os argumentos e retorna a accuracy em porcentagem.
    """
    if model is None:
        model = RandomForestClassifier(n_estimators=300, random_state=random_state, n_jobs=-1)

    if target_col not in data.columns:
        raise ValueError(f"target_col '{target_col}' não encontrado em data.")

    # Define X e y
    drop_cols = [target_col]
    if id_cols:
        drop_cols += [c for c in id_cols if c in data.columns]
    X = data.drop(columns=drop_cols, errors="ignore")
    y = data[target_col]

    # Separa colunas numéricas e categóricas
    numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
    categorical_features = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

    # Split (estratificado quando possível)
    stratify = y if y.nunique() > 1 else None
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify
    )

    # Imputers
    num_imputer = SimpleImputer(strategy=numerical_imputer)
    cat_imputer = SimpleImputer(strategy=categorical_imputer)

    # Encoder
    if encoder.lower() in ("onehot", "one-hot", "ohe"):
        cat_encoder = _make_ohe()
    elif encoder.lower() in ("label", "ordinal"):
        cat_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    else:
        raise ValueError("Parâmetro 'encoder' deve ser 'onehot' ou 'label'.")

    # Pipelines por tipo
    from sklearn.pipeline import Pipeline as SkPipeline
    num_pipe = SkPipeline(steps=[("imputer", num_imputer)])
    cat_pipe = SkPipeline(steps=[("imputer", cat_imputer), ("encoder", cat_encoder)])

    # ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_pipe, numeric_features),
            ("cat", cat_pipe, categorical_features),
        ],
        remainder="drop",
    )

    # Pipeline final
    pipe = Pipeline(steps=[("preprocess", preprocessor), ("model", model)])

    # Treino
    pipe.fit(X_train, y_train)

    # Predição e avaliação
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return float(acc * 100.0)

**Exmplo de uso:**

**Para o train_data:**

In [ ]:
# 1) Garantir que os nulos de Age e Embarked foram restaurados
# df_train = restore_columns_from_original(df_train, df_train_original, ("Age", "Embarked"), key_col="PassengerId")

# 2) Rodar com One-Hot Encoder e imputação mediana para numéricos
from sklearn.ensemble import RandomForestClassifier

acc_pct_ohe = train_eval_pipeline(
    data=train_data,
    encoder="onehot",
    model=RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    numerical_imputer="median",
    categorical_imputer="most_frequent",  # conforme o enunciado
    target_col="Survived",
    id_cols=("PassengerId",),
    test_size=0.2,
    random_state=42,
)
print(f"Acurácia (One-Hot): {acc_pct_ohe:.2f}%")

# 3) Rodar com “label” (OrdinalEncoder) e imputação pela média
acc_pct_label = train_eval_pipeline(
    data=train_data,
    encoder="label",
    model=RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    numerical_imputer="mean",
    categorical_imputer="most_frequent",
    target_col="Survived",
    id_cols=("PassengerId",),
    test_size=0.2,
    random_state=42,
)
print(f"Acurácia (Label/Ordinal): {acc_pct_label:.2f}%")

**3. Aqui serão feitas duas listas, uma com algumas estratégias de imputing de valores
 nulos e outra com os encoders que foram utilizados até agora. O label encoder deve
 ser substituído pelo ordinal encoder, ambos têm o mesmo resultado, contudo o ordinal
 é feito para decodificar features, e o label, para o target. Acesse estas referências
 para entender sobre a utilização do Simple Imputer e Ordinal encoder**

  **a. Definir as 4 seguintes estratégias para imputing:**
 
 **i.Mean: substitui a o valor nulo pela média**
 
 **ii.Most_frequent: substitui pelo valor mais frequente na feature**
 
 **iii.Median: substitui pela mediana**
 
 **iv.Zero: substitui todos os valores nulos por 0**

In [ ]:
from sklearn.impute import SimpleImputer

# Estratégia 1: Média
imputer_mean = SimpleImputer(strategy="mean")

# Estratégia 2: Mais frequente
imputer_most_freq = SimpleImputer(strategy="most_frequent")

# Estratégia 3: Mediana
imputer_median = SimpleImputer(strategy="median")

# Estratégia 4: Constante (zero)
imputer_zero = SimpleImputer(strategy="constant", fill_value=0)

 **b. Definir as 2 estratégias para encoding:**
 **i.One-Hot encoder**
 **ii.Ordinal encoder**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Estratégia 1: One-Hot Encoder
onehot_encoder = OneHotEncoder(handle_unknown="ignore")

# Estratégia 2: Ordinal Encoder
ordinal_encoder = OrdinalEncoder()

**4. A partir da função de criação de pipelines e das listas de encoders e imputers, criar
 um loop que produza todas as combinações possíveis dessas estratégias, e mostre o
 resultado da avaliação de precisão de cada pipeline produzida no loop. (OBS: terá que
 ser usado um “for” dentro de outro).**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler


# 🧠 Defina alvo e separa X e y
target_col = "Survived"
X = train_data.drop(columns=[target_col], errors="ignore")
y = train_data[target_col]

# 🔍 Detecta colunas numéricas e categóricas
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

# ⚙️ Estratégias de imputação numérica
num_imputers = [
    ("mean", SimpleImputer(strategy="mean")),
    ("median", SimpleImputer(strategy="median")),
    ("most_frequent", SimpleImputer(strategy="most_frequent")),
    ("zero", SimpleImputer(strategy="constant", fill_value=0)),
]

# 🧩 Estratégias de codificação
encoders = [
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)),
    ("ordinal", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
]


# ⚖️ Validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 📊 Armazenar resultados
results = []

# 🔁 Loop de todas as combinações
for imp_name, imp in num_imputers:
    num_pipe = Pipeline([
        ("imputer", imp),
        ("scaler", StandardScaler()),
    ])

    for enc_name, enc in encoders:
        cat_pipe = Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("encoder", enc),
        ])

        preprocessor = ColumnTransformer(transformers=[
            ("num", num_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ])

        model = LogisticRegression(
            solver="lbfgs",
            max_iter=5000,   # <- aumenta iterações
            C=0.5,           # <- mais regularização (experimente 0.1 também)
            random_state=42
        )

        pipeline = Pipeline([
            ("prep", preprocessor),
            ("model", model),
        ])

        scores = cross_val_score(pipeline, X, y, cv=cv, scoring="accuracy")
        results.append({
            "Imputação Numérica": imp_name,
            "Codificador": enc_name,
            "Acurácia Média": round(scores.mean() * 100, 2),
            "Desvio Padrão": round(scores.std() * 100, 2)
        })


# 📋 Exibir resultados em ordem
results_df = pd.DataFrame(results).sort_values(by="Acurácia Média", ascending=False)
print("🏆 Ranking de Precisão por Estratégia:")
print(results_df.reset_index(drop=True))

**5. Avalie se alguma dessas estratégias usadas separadamente, gerou uma precisão do
 modelo maior que suas utilizações mescladas, que foi a estratégia realizada para
 produzir o modelo antes das pipelines.**

In [ ]:
#Modelo simples (baseline), sem pipeline 
# ⚙️ Imputação numérica
X_num = X[num_cols].fillna(0)  # imputação zero
X_num = StandardScaler().fit_transform(X_num)

# 🧩 Codificação categórica
X_cat = X[cat_cols].fillna("missing")
encoder = OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)
X_cat_encoded = encoder.fit_transform(X_cat)

# 🔗 Junta tudo
X_final = np.hstack([X_num, X_cat_encoded])

# 🎯 RegLog sem pipeline
model = LogisticRegression(max_iter=5000, C=0.5, random_state=42)
scores = cross_val_score(model, X_final, y, cv=cv, scoring="accuracy")

print("⚖️ Acurácia (modelo simples):", round(scores.mean() * 100, 2), "+/-", round(scores.std() * 100, 2))

**Como o modelo simples teve a mesma acurácia da Pipeline isso levanta a questão se vale a pena manter o modelo com a pipeline. A resposta é sim, pois ele possui vantagens que o outro não tem, vatagens essas que são as seguintes:**

**Organização: deixa o código replicável e lipo**
**Reprodutibilidade: evita esquecimentos em etapas de pré-processamento.**
**Validação correta: garante que transformações sejam feitas dentro dos folds, prevenindo data leakage.**
**Escalabilidade: se você mudar para outro modelo ou aplicar grid search, é só plugar no pipeline.**



****

**8.3- Cross Validation e Gradient Boosting**

**1. O cross validation é uma maneira melhor para avaliar os modelos com os quais
 estamos lidando. Portanto, o modelo que melhor se saiu nos teste até agora, deverá
 ser avaliado utilizando este método. Como resultado será utilizada a média entre as
 avaliações, lembre-se de utilizar o scoring = “accuracy”.**

O modelo que apresentou melhor desempenho foi o de imputação numérica zero, codificador: one hot,acuracia média: 82.16%, desvio padrão: 1.55.

**2. Assim como o Random Forest, o XGBoost também tem seu equivalente como
 classifier. O Gradient Boosting Classifier será utilizado agora para gerar um novo
 modelo, a partir do dataframe que foi manipulado com as melhores estratégias até
 agora.**

 **a. Importar o Gradient Boosting Classifier, utilizando random state = 0 e
 n_iter_no_change = 100 (Entender o que são esses parâmetros a partir da
 documentação do algoritmo);**

 **b. Determinar as features e o target que serão utilizados para gerar o modelo;**

 **c. Treinar e avaliar o modelo utilizando cross validation;**

 **d. A partir da nota gerada pela avaliação, determinar qual algoritmo se saiu
 melhor, Random Forest ou Gradient Boosting.**

**Importar o modelo e entender os parâmetros:**

In [31]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

**Definir features e target**

In [32]:
# Definir variáveis
target_col = "Survived"
drop_cols = ["PassengerId", "Name", "Ticket", "Cabin"]  # você pode ajustar conforme seu dataset

# Separar X e y
X = train_data.drop(columns=[target_col] + drop_cols, errors="ignore").copy()
y = train_data[target_col].copy()

# Codificar variáveis categóricas com OneHotEncoder
obj_cols = X.select_dtypes(include="object").columns.tolist()

if obj_cols:
    print("🔍 Categóricas encontradas:", obj_cols)
    encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
    encoded = encoder.fit_transform(X[obj_cols])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(obj_cols), index=X.index)
    X = pd.concat([X.drop(columns=obj_cols), encoded_df], axis=1)

# Imputar nulos
X = pd.DataFrame(SimpleImputer(strategy="median").fit_transform(X), columns=X.columns, index=X.index)

🔍 Categóricas encontradas: ['Sex', 'Embarked']


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


**Treinar e avaliar com cross-validation:**

In [33]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = "roc_auc"

gb = GradientBoostingClassifier(
    random_state=0,
    n_iter_no_change=100,
    validation_fraction=0.15
)

rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=1,
    random_state=0,
    n_jobs=-1
)

scores_gb = cross_val_score(gb, X, y, cv=cv, scoring=scoring, n_jobs=-1)
scores_rf = cross_val_score(rf, X, y, cv=cv, scoring=scoring, n_jobs=-1)

print(f"🌲 Random Forest AUC => média: {scores_rf.mean():.4f} ± {scores_rf.std():.4f}")
print(f"🎯 Gradient Boosting AUC => média: {scores_gb.mean():.4f} ± {scores_gb.std():.4f}")

🌲 Random Forest AUC => média: 0.8690 ± 0.0264
🎯 Gradient Boosting AUC => média: 0.8616 ± 0.0366


**Comparar com RandoForest e dizer qual foi melhor:**

In [34]:
# Escolher o melhor modelo
melhor_est = gb if scores_gb.mean() >= scores_rf.mean() else rf
melhor_nome = "Gradient Boosting" if scores_gb.mean() >= scores_rf.mean() else "Random Forest"
print(f"💡 Modelo vencedor: {melhor_nome}")
melhor_est.fit(X, y)

💡 Modelo vencedor: Random Forest


RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)

**Para o test_data:**

In [36]:
# Preparar test_data (replicar mesmo tratamento de X)
X_test = test_data.drop(columns=drop_cols, errors="ignore").copy()

# Codificar categóricas
obj_test_cols = X_test.select_dtypes(include="object").columns.tolist()

if obj_test_cols:
    encoded_test = encoder.transform(X_test[obj_test_cols])
    encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(obj_test_cols), index=X_test.index)
    X_test = pd.concat([X_test.drop(columns=obj_test_cols), encoded_test_df], axis=1)

# Imputar nulos
X_test = pd.DataFrame(SimpleImputer(strategy="median").fit_transform(X_test), columns=X_test.columns, index=X_test.index)

# Alinhar colunas com treino
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Previsão
preds_test = melhor_est.predict_proba(X_test)[:, 1]